<a href="https://colab.research.google.com/github/onurkasap/Computer-Vision/blob/main/sam2_rally_car_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 67.9 MB/s eta 0:00:00


#  GEREKLİ KÜTÜPHANELER


In [2]:
from ultralytics import SAM
import cv2
from google.colab import files
from IPython.display import Video, display
import os

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
if not os.path.exists('/content/video.mp4'):
    print("📤 Video dosyanızı yükleyin...")
    uploaded = files.upload()
    video_path = list(uploaded.keys())[0]
    print(f"✅ Video yüklendi: {video_path}")
else:
    video_path = '/content/video.mp4'
    print(f"✅ Video bulundu: {video_path}")


✅ Video bulundu: /content/video.mp4


# SAM 2 MODELİNİ YÜKLE

In [6]:
print("\n🤖 SAM 2 modeli yükleniyor...")
print("   (İlk seferde model indirilecek, ~150MB)")

model = SAM("sam2.1_b.pt")
print("✅ Model yüklendi!")


🤖 SAM 2 modeli yükleniyor...
   (İlk seferde model indirilecek, ~150MB)
✅ Model yüklendi!


# SEGMENTASYON YAP VE VIDEO OLARAK KAYDET

In [7]:
print("\n🎬 Segmentasyon başlıyor...")
print("   (Bu işlem biraz zaman alabilir)")

# Output dosya adı
output_video = '/content/segmented_output.mp4'

# Video writer ayarla
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

# Segmentasyon yap
results = model(video_path, stream=True, verbose=False)

frame_count = 0
object_counts = []

for r in results:
    frame_count += 1

    # Segmente edilmiş frame'i al
    annotated_frame = r.plot()

    # Video'ya yaz
    out.write(annotated_frame)

    # Obje sayısını kaydet
    if r.masks is not None:
        num_objects = len(r.masks)
        object_counts.append(num_objects)
    else:
        object_counts.append(0)

    # İlerleme göster
    if frame_count % 30 == 0:
        progress = (frame_count / total_frames) * 100
        avg_objects = sum(object_counts) / len(object_counts)
        print(f"   İşlenen: {frame_count}/{total_frames} "
              f"({progress:.1f}%) | Ort. Obje: {avg_objects:.1f}",
              end='\r')

# Video'yu kapat
out.release()

print(f"\n\n✅ SEGMENTASYON TAMAMLANDI!")
print("=" * 50)



🎬 Segmentasyon başlıyor...
   (Bu işlem biraz zaman alabilir)


✅ SEGMENTASYON TAMAMLANDI!


# İSTATİSTİKLER


In [8]:
import numpy as np

print("\n📊 İSTATİSTİKLER:")
print("=" * 50)
print(f"İşlenen Frame     : {frame_count}")
print(f"Ortalama Obje/Frame: {np.mean(object_counts):.2f}")
print(f"Min Obje         : {min(object_counts)}")
print(f"Max Obje         : {max(object_counts)}")
print(f"Output Dosya     : {output_video}")
print("=" * 50)


📊 İSTATİSTİKLER:
İşlenen Frame     : 47
Ortalama Obje/Frame: 6.55
Min Obje         : 2
Max Obje         : 11
Output Dosya     : /content/segmented_output.mp4
